<table>
    <tr>
        <td><img src="./img/Macc.png" width="auto"/></td>
        <td>
            <table><tr>
            <h1 style="color:blue;text-align:center">Lógica para Ciencias de la Computación</h1></td>
            </tr></table>   
        <td>&nbsp;</td>
        <td>
            <table><tr>
            <tp><p style="font-size:150%;text-align:center">Taller</p></tp>
            <tp><p style="font-size:150%;text-align:center">Implementación de Tableaux</p></tp>
            </tr></table>
        </td>
    </tr>
</table>

---

# Objetivos <a class="anchor" id="inicio"></a>

El objetivo de este notebook es implementar el algoritmo de construcción de tableros semánticos (tableaux). El método de los tableaux es un SATsolver, el cual recibe una fórmula y devuelve una interpretación de la misma, si es que existe, o informa que la fórmula es insatisfacible. Veremos diferentes maneras de implementar este método, dependiendo de distintas maneras de expandir el árbol del tableaux.


# Secciones

1. [Clasificación de fórmulas para tableaux.](#clas)
2. [Expansión primero en anchura.](#anchura)
3. [Expansión primero en profundidad.](#prof)
4. [Recorrido backtracking.](#backt)
5. [Eficiencia de los algoritmos (opcional).](#efi)

# Clasificación de fórmulas para tableaux <a class="anchor" id="clas"></a>

([Volver al inicio](#inicio))

Recordemos el siguiente diagrama de flujo para la implementación de un tableaux $\tau$ para una fórmula $A$:

<img src="./img/flow.png" width="auto"/>

Hay tres elementos centrales en este algoritmo:

* Nodos.
* Selección del próximo nodo a expandir.
* Clasificación de una fórmula como $\alpha$ o $\beta$.

En esta sección veremos la clasificación de fórmulas y la implementación de los nodos. En las subsiguientes secciones veremos tres maneras distintas de seleccionar el próximo nodo a expandir.

### Clasificación de una fórmula como $\alpha$ o $\beta$

Para realizar la implementación de los tableaux, el orden más apropiado es primero implementar la clasificación de fórmulas. Hemos visto que toda fórmula o es un literal, o puede clasificarse como una fórmula de tipo $\alpha$ o $\beta$. Para referencia, repetimos las clasificaciones en la siguiente figura:

<img src="./img/clasif.png" width="200px"/>

En el siguiente código hemos implementado la clasificación para dos casos: literales y fórmulas 1$\alpha$:

In [3]:
from Logica import *
from itertools import product
from copy import deepcopy

class Formula :

    def __init__(self) :
        pass

    def __str__(self) :
        if type(self) == Letra:
            return self.letra
        elif type(self) == Negacion:
            return '-' + str(self.subf)
        elif type(self) == Binario:
            return "(" + str(self.left) + self.conectivo + str(self.right) + ")"

    def letras(self):
        if type(self) == Letra:
            return set(self.letra)
        elif type(self) == Negacion:
            return self.subf.letras()
        elif type(self) == Binario:
            return self.left.letras().union(self.right.letras())

    def subforms(self):
        if type(self) == Letra:
            return [str(self)]
        elif type(self) == Negacion:
            return list(set([str(self)] + self.subf.subforms()))
        elif type(self) == Binario:
            return list(set([str(self)] + self.left.subforms() + self.right.subforms()))

    def valor(self, I) :
        if type(self) == Letra:
            return I[self.letra]
        elif type(self) == Negacion:
            return not self.subf.valor(I)
        elif type(self) == Binario:
            if self.conectivo == 'Y':
                return self.left.valor(I) and self.right.valor(I)
            if self.conectivo == 'O':
                return self.left.valor(I) or self.right.valor(I)
            if self.conectivo == '>':
                return not self.left.valor(I) or self.right.valor(I)
            if self.conectivo == '=':
                return (self.left.valor(I) and self.right.valor(I)) or (not self.left.valor(I) and not self.right.valor(I))

    def SATtabla(self):
        letras = list(self.letras())
        n = len(letras)
        valores = list(product([True, False], repeat=n))
        for v in valores:
            I = {letras[x]: v[x] for x in range(n)}
            if self.valor(I):
                return I
        return None

class Letra(Formula) :
    def __init__ (self, letra:str) :
        self.letra = letra

class Negacion(Formula) :
    def __init__(self, subf:Formula) :
        self.subf = subf

class Binario(Formula) :
    def __init__(self, conectivo:str, left:Formula, right:Formula) :
        assert(conectivo in ['Y','O','>','='])
        self.conectivo = conectivo
        self.left = left
        self.right = right

def inorder_to_tree(cadena:str):
    conectivos = ['Y', 'O', '>', '=']
    if len(cadena) == 1:
        return Letra(cadena)
    elif cadena[0] == '-':
        return Negacion(inorder_to_tree(cadena[1:]))
    elif cadena[0] == "(":
        counter = 0 #Contador de parentesis
        for i in range(1, len(cadena)):
            if cadena[i] == "(":
                counter += 1
            elif cadena[i] == ")":
                counter -=1
            elif cadena[i] in conectivos and counter == 0:
                return Binario(cadena[i], inorder_to_tree(cadena[1:i]),inorder_to_tree(cadena[i + 1:-1]))
    else:
        raise Exception('¡Cadena inválida!')

class nodos_tableaux:

    def __init__(self, fs):
        clasfs = [(A, str(A), *A.clasifica_para_tableaux()) for A in fs]
        self.alfas = [c for c in clasfs if c[3] == 'alfa']
        self.betas = [c for c in clasfs if c[3] == 'beta']
        self.literales = [c for c in clasfs if c[3] == 'literal']

    def __str__(self):
        cadena = f'Alfas:{[str(c[1]) for c in self.alfas]}\n'
        cadena += f'Betas:{[str(c[1]) for c in self.betas]}\n'
        cadena += f'Literales:{[str(c[1]) for c in self.literales]}'
        return cadena

    def tiene_lit_comp(self):
        lits = [c[1] for c in self.literales]
        l_pos = [l for l in lits if '-' not in l]
        l_negs = [l[1:] for l in lits if '-' in l]
        return len(set(l_pos).intersection(set(l_negs))) > 0

    def es_hoja(self):
        if self.tiene_lit_comp():
            return 'cerrada'
        elif ((len(self.alfas) == 0) and (len(self.betas) == 0)):
            return 'abierta'
        else:
            return None

    def interp(self):
        I = {}
        for lit in self.literales:
            l = lit[1]
            if '-' not in l:
                I[l] = True
            else:
                I[l[1:]] = False
        return I

    def expandir(self):
        '''Escoge última alfa, si no última beta, si no None'''
        f_alfas = deepcopy(self.alfas)
        f_betas = deepcopy(self.betas)
        f_literales = deepcopy(self.literales)
        if len(self.alfas) > 0:
            f, s, num_regla, cl = f_alfas.pop(0)
            if num_regla == 1:
                formulas = [f.subf.subf]
            elif num_regla == 2:
                formulas = [f.left, f.right]
            elif num_regla == 3:
                formulas = [Negacion(f.subf.left), Negacion(f.subf.right)]
            elif num_regla == 4:
                formulas = [f.subf.left, Negacion(f.subf.right)]
            for nueva_f in formulas:
                clasf = nueva_f.clasifica_para_tableaux()
                if clasf[1]== 'alfa':
                    lista = f_alfas
                elif clasf[1]== 'beta':
                    lista = f_betas
                elif clasf[1]== 'literal':
                    lista = f_literales
                strs = [c[1] for c in lista]
                if str(nueva_f) not in strs:
                    lista.append((nueva_f, str(nueva_f), *clasf))
            nuevo_nodo = nodos_tableaux([])
            nuevo_nodo.alfas = f_alfas
            nuevo_nodo.betas = f_betas
            nuevo_nodo.literales = f_literales
            return [nuevo_nodo, None]
        elif len(self.betas) > 0:
            f, s, num_regla, cl = f_betas.pop(0)
            if num_regla == 1:
                B1 = Negacion(f.subf.left)
                B2 = Negacion(f.subf.right)
            elif num_regla == 2:
                B1 = f.left
                B2 = f.right
            elif num_regla == 3:
                B1 = Negacion(f.left)
                B2 = f.right
            f_alfas2 = deepcopy(f_alfas)
            f_betas2 = deepcopy(f_betas)
            f_literales2 = deepcopy(f_literales)
            clasf = B1.clasifica_para_tableaux()
            if clasf[1]== 'alfa':
                lista = f_alfas
            elif clasf[1]== 'beta':
                lista = f_betas
            elif clasf[1]== 'literal':
                lista = f_literales
            strs = [c[1] for c in lista]
            if str(B1) not in strs:
                lista.append((B1, str(B1), *clasf))
            clasf = B2.clasifica_para_tableaux()
            if clasf[1]== 'alfa':
                lista = f_alfas2
            elif clasf[1]== 'beta':
                lista = f_betas2
            elif clasf[1]== 'literal':
                lista = f_literales2
            strs = [c[1] for c in lista]
            if str(B2) not in strs:
                lista.append((B2, str(B2), *clasf))
            n1 = nodos_tableaux([])
            n1.alfas = f_alfas
            n1.betas = f_betas
            n1.literales = f_literales
            n2 = nodos_tableaux([])
            n2.alfas = f_alfas2
            n2.betas = f_betas2
            n2.literales = f_literales2
            return [n1, n2]
        else:
            return [None, None]




In [22]:
def clasifica_para_tableaux(self):
    if type(self) == Letra:
        return None, 'literal'          # Literal positivo
    elif type(self) == Negacion:
        if type(self.subf) == Letra:
            return None, 'literal'      # Literal negativo
        elif type(self.subf) == Negacion:
            return 1, 'alfa'            # Fórmula 1 alfa
        elif type(self.subf) == Binario:
            if self.subf.conectivo == 'O':
                return 3, 'alfa'
            elif self.subf.conectivo == 'Y':
                return 1, 'beta'
            elif self.subf.conectivo == '>':
                return 4, 'alfa'
    elif type(self) == Binario:
        if self.conectivo == 'Y':
            return 2, 'alfa'
        elif self.conectivo == 'O':
            return 2, 'beta'
        elif self.conectivo == '>':
            return 3, 'beta'
        
setattr(Formula,"clasifica_para_tableaux",clasifica_para_tableaux)

**Ejercicio 1:**

Complete la anterior función `clasifica_para_tableaux` para clasificar el resto de fórmulas de acuerdo a su tipo. Pruebe su implementación con las siguientes fórmulas:

In [23]:
f = '--(-(pOq)Y-(r>s))'  # 1, alfa
A = inorder_to_tree(f)
print(A.clasifica_para_tableaux())

f = '(-(pOq)Y-(r>s))'  # 2, alfa
A = inorder_to_tree(f)
print(A.clasifica_para_tableaux())

f = '-(pOq)'  # 3, alfa
A = inorder_to_tree(f)
print(A.clasifica_para_tableaux())

f = '-(r>s)' # 4, alfa
A = inorder_to_tree(f)
print(A.clasifica_para_tableaux())

f = '-(pYq)' # 1, beta
A = inorder_to_tree(f)
print(A.clasifica_para_tableaux())

f = '(-(pYq)O(r>s))' # 2, beta
A = inorder_to_tree(f)
print(A.clasifica_para_tableaux())

f = '(r>s)' # 3, beta
A = inorder_to_tree(f)
print(A.clasifica_para_tableaux())

(1, 'alfa')
(2, 'alfa')
(3, 'alfa')
(4, 'alfa')
(1, 'beta')
(2, 'beta')
(3, 'beta')


---

### Implementación de los nodos

El tableaux que vamos a implementar es un árbol basado en nodos. Los nodos son de la clase `nodos_tableaux` que se encuentra en la librería `Logica`. La descripción de la clase es la siguiente:

**Atributos:**

* `alfas`: una lista con las fórmulas tipo $\alpha$, representadas como una 4-tupla que incluye la fórmula como árbol, como cadena, el número de la regla y la cadena 'alfa'.
* `betas`: una lista con las fórmulas tipo $\beta$, representadas como una 4-tupla que incluye la fórmula como árbol, como cadena, el número de la regla y la cadena 'beta'.
* `literales`: una lista con los literales, representados como una 4-tupla que incluye la fórmula como árbol, como cadena, None y la cadena 'literal'.

**Métodos:**

* `tiene_lit_comp`: retorna `True` si self.`literales` tiene un par complementario de literales.
* `es_hoja`: retorna 'Cerrada' si self.`literales` tiene un par complementario de literales; 'Abierta' si self.`literales` NO tiene un par complementario de literales y no tiene reglas ni alfa ni beta; None en otro caso.
* `interp`: retorna un diccionario que hace veradero a todos los literales en self.`literales`.
* `expandir`: retorna un `nodo_tableaux` que es el resultado de aplicar la primera regla $\alpha$. Si no hay reglas $\alpha$, retorna dos `nodo_tableaux` que son el resultado de aplicar la primera regla $\beta$. Si no hay reglas $\beta$, retorna None, None.

Para ilustrar esta clase, instanciemos un objeto a partir de una fórmula $\alpha$, una $\beta$ y un literal:

In [27]:
print('Nodo n:')
forms = ['-(p>q)', '(pOq)', '-p']
forms = [inorder_to_tree(f) for f in forms]
n = nodos_tableaux(forms)
print(n)
print('')
print('¿Tiene literales complementarios?', n.tiene_lit_comp())
print('¿Es una hoja?', n.es_hoja())
print('Interpretación para literales:', n.interp())

Nodo n:
Alfas:['-(p>q)']
Betas:['(pOq)']
Literales:['-p']

¿Tiene literales complementarios? False
¿Es una hoja? None
Interpretación para literales: {'p': False}


El método `expandir` tomará la primera (y única en este caso) fórmula $\alpha$ y devolvera una lista [`None`, `n1`] donde `n1` es el nodo resultado de aplicar la regla:

In [38]:
print(n)
hijos = n.expandir()
n1= hijos[1]
n1.expadir()

Alfas:['-(p>q)']
Betas:['(pOq)']
Literales:['-p']


AttributeError: 'NoneType' object has no attribute 'expadir'

In [26]:
for hijo in hijos:
    if hijo != None:
        print(hijo)

Alfas:[]
Betas:['(pOq)']
Literales:['-p', 'p', '-q']


**Ejercicio 2:**

Explanda el hijo de `n` y verifique si los nodos resultado tienen pares complementarios de literales.

---

**Ejercicio 3:**

Expanda manualmente (o mediante un while) el siguiente nodo hasta obtener solo una lista de literales y devuelva la interpretación que los hace verdaderos. Observe que todas sus subfórmulas son de tipo $\alpha$:

In [9]:
forms = ['--(-(pOq)Y-(r>s))']
forms = [inorder_to_tree(f) for f in forms]
n = nodos_tableaux(forms)

---

# Expansión primero en anchura <a class="anchor" id="anchura"></a>

([Volver al inicio](#inicio))

Ya tenemos los elementos necesarios para la generación del tableaux, pero necesitamos especificar una manera de ir expandiendo el árbol. Además de la opción de expandirlo de manera aleatoria -- seleccionando aleatoriamente un nodo que no sea una hoja -- tenemos tres opciones. En esta sección examinaremos la expansión primero en anchura. El pseudocódigo del algoritmo es el siguiente:

<img src="./img/anchura.png" width="350px"/>

La implementación en Python es la siguiente:

In [10]:
def primero_anchura(nodo):
    estado = nodo
    res = estado.es_hoja()
    if res == 'cerrada':
        return None
    elif res == 'abierta':
        return estado.interp()
    frontera = [estado]
    while len(frontera) > 0:
        estado = frontera.pop(0) 
        hijos = estado.expandir()
        for a in hijos:
            if a != None:
                res = a.es_hoja()
                if res == 'abierta':
                    return a.interp()
                elif res == None:
                    frontera.append(a)
    return None

Podemos usar esta función para expandir el nodo del ejercicio 3:

In [11]:
forms = ['--(-(pOq)Y-(r>s))']
forms = [inorder_to_tree(f) for f in forms]
n = nodos_tableaux(forms)
I = primero_anchura(n)
print("Resultado:", I)

AttributeError: 'Binario' object has no attribute 'subf'

**Ejercicio 4:**

Expanda el nodo correspondiente a la siguiente fórmula:

'(((pYq)O(rOq))Y(-qO-p))'

Debe obtener la siguiente interpretación:

Resultado: {'q': False, 'r': True}

---

# Expansión primero en profundidad <a class="anchor" id="prof"></a>

([Volver al inicio](#inicio))

Además de poder expandir primero en anchura, podemos expandir primero en profundidad. El pseudocódigo de este algoritmo es el siguiente:

<img src="./img/profundidad.png" width="350px"/>

**Ejercicio 5:**

Implemente la función `primero_profundidad` y expanda los nodos para las siguientes fórmulas:

* '--(-(pOq)Y-(r>s))'
* '(((pYq)O(rOq))Y(-qO-p))'

Las soluciones deben ser las siguientes:

* Resultado: {'p': False, 'q': False, 'r': True, 's': False}
* Resultado: {'p': False, 'r': True}

---

# Recorrido con backtracking <a class="anchor" id="backt"></a>

([Volver al inicio](#inicio))

Podemos hacer también un recorrido por el árbol haciendo una expansión recursiva que implementa el backtracking, como en el siguiente pseudocódigo:

<img src="./img/backtracking.png" width="350px"/>

**Ejercicio 6:**

Implemente la función `backtracking` y expanda los nodos para las siguientes fórmulas:

* '--(-(pOq)Y-(r>s))'
* '(((pYq)O(rOq))Y(-qO-p))'

Las soluciones deben ser las siguientes:

* Resultado: {'p': False, 'q': False, 'r': True, 's': False}
* Resultado: {'q': False, 'r': True}

---

# Sección opcional: Eficiencia de los algoritmos <a class="anchor" id="efi"></a>


([Volver al inicio](#inicio))

Vamos a verificar empíricamente cuál función es más rápida para expandir el árbol del tableaux de una fórmula dada. Para ello, crearemos fórmulas más y más grandes para probar en ellas nuestros tres algoritmos.

Definimos nuestras funciones para medir tiempos en una lista de entradas: 

In [12]:
from time import time
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

def obtiene_tiempos(fun, args, num_it=100):
    tiempos_fun = []
    for i in range(num_it):
        arranca = time()
        x = fun(*args)
        para = time()
        tiempos_fun.append(para - arranca)
    return tiempos_fun

def compara_entradas_funs(funs, nombres_funs, lista_args, N=100):
    entradas = []
    funcion = []
    tiempos = []
    lista_dfs = []
    for i, args in enumerate(lista_args):
        for j, fun in enumerate(funs):
            t = obtiene_tiempos(fun, [args], N)
            tiempos += t
            n = len(t)
            entradas += [i+1]*n
            funcion += [nombres_funs[j]]*n
        df = pd.DataFrame({'Long_entrada':entradas, 
                           'Funcion':funcion,
                           'Tiempo_prom':tiempos})
        lista_dfs.append(df)
    df = pd.concat(lista_dfs)
    sns.lineplot(x='Long_entrada',y='Tiempo_prom',hue='Funcion',data=df)
    plt.show()

ModuleNotFoundError: No module named 'seaborn'

Creamos nuestra lista de fórmulas:

In [13]:
from random import uniform, choice

cantidad = 20
letras = [chr(i) for i in range(256, 256+cantidad)]
lista = [letras[0]]
formula = letras[0]
for p in letras[1:]:
    neg1 = '-' if uniform(0,1) > .5 else ''
    neg2 = '-' if uniform(0,1) > .5 else ''
    conectivo = choice(['Y','O','>'])
    formula = neg2 + "(" + formula + conectivo + neg1 + p + ")"
    lista.append(formula)

#lista

Y, finalmente, medimos los tiempos de ejecución para cada fórmula:

In [14]:
nombres = ['primero anchura', 'primero profundidad', 'backtracking']
anchura = lambda formula: primero_anchura(nodos_tableaux([inorder_to_tree(formula)]))
profundidad = lambda formula: primero_profundidad(nodos_tableaux([inorder_to_tree(formula)]))
backtrack = lambda formula: backtracking(nodos_tableaux([inorder_to_tree(formula)]))
funs = [anchura, profundidad, backtrack]
compara_entradas_funs(funs, nombres, lista)

NameError: name 'compara_entradas_funs' is not defined

En esta gráfica es muy fácil apreciar que, a medida que aumentamos el número de letras proposicionales distintas en la fórmula de entrada, los tres algoritmos se tardan más en determinar si ella es satisfacible. De los tres algoritmos, el peor es el de backtracking y el mejor es el de primero en profundidad. No obstante, vemos que el número de letras proposicionales no es el único factor que influye en el tiempo de ejecución. También influye la estructura de la fórmula. Más adelante estudiaremos el comportamiento de los algoritmos de acuerdo a distintos tipos de fórmulas.

---